### Imports

In [ ]:
from pmaw import PushshiftAPI
# from psaw import PushshiftAPI
from psycopg2.extras import Json
from tqdm.auto import tqdm
import datetime
import pandas as pd
import numpy as np 
import requests
import json
import praw
import psycopg2
from psycopg2.extras import execute_values, execute_batch
import datetime
import time
import re

from reddit_stuff import backfill_submissions

if "insert_all_comments_to_db_pmaw" in locals(): 
    del insert_all_comments_to_db_pmaw

from reddit_stuff import insert_all_comments_to_db_pmaw

# import spacy
# from spacymoji import Emoji
# from spacy import displacy

pmaw_api = PushshiftAPI()

import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter
%matplotlib inline

### Functions

In [ ]:
# Conn 
if "get_conn" in locals():
    del get_conn
    
def get_conn() -> psycopg2.connect:
    return psycopg2.connect(
        dbname="polygonio", 
        port="5433", 
        host="localhost", 
        password='rogerthat', 
        user="postgres"
    )

In [ ]:
query = "SELECT 1;"
with get_conn() as conn:
    with conn.cursor() as cur:
        cur.execute(query)
        res = cur.fetchall()

### Get all comments from pmaw

In [ ]:
if "get_unique_col_from_db" in locals(): 
    del get_unique_col_from_db
    
def get_unique_col_from_db(colname:str) -> list: 
    with get_conn() as conn: 
        with conn.cursor() as cur:
            query = f"SELECT DISTINCT({colname}) FROM wsb_comments;"
            cur.execute(query)
            results = cur.fetchall()
    results = [i[0] for i in results]
    return results

In [ ]:
all_submission_ids = get_unique_col_from_db(colname="submission_id")
all_comment_ids_in_db = get_unique_col_from_db(colname="id")

In [ ]:
# submission_comments = pd.read_csv("all_submissions_and_comments.csv", index_col=0)
# submission_comments.loc[:, "comments"] = submission_comments["comments"].apply(lambda x: json.loads(x.replace("'", "\""))["data"] )

In [ ]:
submission_id_comments = {}
for submission_id in tqdm(all_submission_ids):
    submission_id_comments[submission_id] = pmaw_api.search_submission_comment_ids(
        ids=[submission_id], 
        safe_exit=True, 
        memsafe=True
    ) 

In [ ]:
submission_id_comments_to_df = []
for submission_id, comments in tqdm(submission_id_comments.items()):
    submission_id_comments_to_df.append({"submission_id": submission_id, "comments": [comment for comment in comments]})
    
submission_comments = pd.DataFrame(submission_id_comments_to_df)

In [ ]:
all_comments = []
for sub_id in tqdm(submission_comments.submission_id):
    comments = submission_comments.loc[submission_comments.submission_id == sub_id, "comments"].values[0]
    comments_arr = pmaw_api.search_comments(ids=comments)
    for c in comments_arr:
        c["submission_id"] = sub_id
        all_comments.append(c)
        
comments_df = pd.DataFrame(all_comments)

In [ ]:
to_insert_comments = []
for i in tqdm(range(len(comments_df))):
    comment = comments_df.iloc[i, ].to_dict()
    _ = comment.pop("media_metadata")
    comment["subreddit_name_prefixed"] = f"r/{comment['subreddit']}"
    _ = comment.pop("subreddit")

    for key, val in comment.items():
        if key in ["awarders", "user_reports", "all_awardings", "report_reasons", "gildings",
                   "author_flair_richtext", "treatment_tags", "mod_reports"]:
            comment[key] = Json(json.dumps(val))
            
        if isinstance(comment[key], np.bool_):
            comment[key] = bool(val)
            
        if isinstance(comment[key], np.int64):
            comment[key] = int(val)

    if isinstance(comment["edited"], bool):
        comment["edited"] = 0
        
    if isinstance(comment["author_patreon_flair"], bool):
        comment["author_patreon_flair"] = ""
        
    if isinstance(comment["author_cakeday"], np.float): 
        comment["author_cakeday"] = bool(0)
        
    if isinstance(comment["author_premium"], np.float): 
        comment["author_premium"] = bool(0)

    cols_str = ','.join(comment.keys())
    values_str = ('%s,' * len(comment)).rstrip(',')
    insert_query = f"""INSERT INTO wsb_comments ({cols_str}) VALUES ({values_str}) ON CONFLICT (id) DO NOTHING;"""
    to_insert_comments.append(tuple(comment.values()))    

In [ ]:
insert_query = f"INSERT INTO wsb_comments ({cols_str}) VALUES %s ON CONFLICT (id) DO NOTHING;"

In [ ]:
with get_conn() as conn:
    with conn.cursor() as cur:
        try:
            execute_values(cur, insert_query, to_insert_comments)
            conn.commit()
#             mogrified_query = cur.mogrify(insert_query, tuple(comment.values()))
#             cur.execute(mogrified_query)
        except psycopg2.errors.ProgrammingError as e:
            print(e)

In [ ]:
with get_conn() as conn:
    with conn.cursor() as cur:
        cur.execute("SELECT t.submission_ids_completed FROM wsb_comments_status t ORDER BY t.insert_date DESC LIMIT 1;")
        all_submission_ids_completed = cur.fetchall()[0][0]

### Get some submission id's

In [1]:
import requests

In [9]:
url = "https://api.polygon.io/v3/reference/tickers"

resp = requests.get(url, params={
    "active": "true", 
    "limit": 1000, 
    "apikey": "UPUKTfaMrhV1k5ZyCvBUbv_1pAjZ24zkUbLD_T"
})

all_data = []
data = resp.json()
while "next_url" in data.keys():
    resp = requests.get(data["next_url"], params={"apikey": "UPUKTfaMrhV1k5ZyCvBUbv_1pAjZ24zkUbLD_T", "limit": 1000})
    data = resp.json()
    all_data.append(data)

In [19]:
import pandas as pd

In [14]:
import plotly.express as px

df = px.data.stocks(indexed=True)

In [15]:
df

company,GOOG,AAPL,AMZN,FB,NFLX,MSFT
date,,,,,,
2018-01-01,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
2018-01-08,1.018172,1.011943,1.061881,0.959968,1.053526,1.015988
2018-01-15,1.032008,1.019771,1.053240,0.970243,1.049860,1.020524
2018-01-22,1.066783,0.980057,1.140676,1.016858,1.307681,1.066561
2018-01-29,1.008773,0.917143,1.163374,1.018357,1.273537,1.040708
...,...,...,...,...,...,...
2019-12-02,1.216280,1.546914,1.425061,1.075997,1.463641,1.720717
2019-12-09,1.222821,1.572286,1.432660,1.038855,1.421496,1.752239
2019-12-16,1.224418,1.596800,1.453455,1.104094,1.604362,1.784896


In [ ]:
posts = pmaw_api.search_submissions(q="Daily Discussion Thread", subreddit="wallstreetbets", limit=5000)
submissions = [s for s in posts]

In [ ]:
len(submissions)

In [ ]:
exceptions = []
all_dates_gathered = []
for sub in tqdm(submissions):
    if "link_flair_text" in sub.keys():
        if sub["link_flair_text"] == 'Daily Discussion':
            try:
                date_info = pd.to_datetime('-'.join( sub["full_link"].split("/")[-2].split("_")[-4:][1:]), format="%B-%d-%Y")
                all_dates_gathered.append({"date": date_info, "sub_links": []})
            except (TypeError, ValueError) as e: 
                print(e)
                exceptions.append(sub)
        else: 
            pass
    else:
        pass

In [ ]:
final_dates_gathered = []
for sub in tqdm(submissions):
    if "link_flair_text" in sub.keys():
        if sub["link_flair_text"] == 'Daily Discussion':
            try:
                date_info = pd.to_datetime('-'.join(sub["full_link"].split("/")[-2].split("_")[-4:][1:]), format="%B-%d-%Y")
                for ele in all_dates_gathered:
                    if ele["date"] == date_info: 
                        ele["sub_links"].append(sub["full_link"])
                        final_dates_gathered.append(ele)

            except (TypeError, ValueError, AssertionError) as e:
                exceptions.append(date_info)
        else: 
            pass
    else: 
        pass

In [ ]:
all_submissions_meta_df = pd.DataFrame(final_dates_gathered)
all_submissions_meta_df.drop_duplicates(subset=["date"], inplace=True)
all_submissions_meta_df.reset_index(drop=True, inplace=True)
all_submissions_meta_df.set_index("date", inplace=True)
all_submissions_meta_df = all_submissions_meta_df.explode("sub_links")
all_submissions_meta_df = all_submissions_meta_df.reset_index()
all_submissions_meta_df.columns = ["date", "links_in_reddit"]

### Check if these are present in the database, and how many comments do each of these submissions have?
with get_conn() as conn: 
    with conn.cursor() as cur: 
        cur.execute("SELECT DISTINCT id, full_link FROM wsb_submissions WHERE link_flair_text = 'Daily Discussion';")
        sub_in_db_df = pd.DataFrame(cur.fetchall(), columns=["id", "links_in_db"])

merged_df = pd.merge(left=all_submissions_meta_df, right=sub_in_db_df, left_on="links_in_reddit", right_on="links_in_db", how="outer")
in_db_not_daily_subs_df = merged_df[merged_df["date"].isna()].copy()
in_db_possible_backup_needed_df = merged_df[~merged_df["date"].isna()].copy()
in_db_possible_backup_needed_df = in_db_possible_backup_needed_df.sort_values(by="date")
in_db_possible_backup_needed_df = in_db_possible_backup_needed_df.reset_index(drop=True)

# Grab the submissions that are not there in db and push them
submission_links_not_uploaded = \
set(in_db_possible_backup_needed_df["links_in_reddit"].values.tolist()) - \
set(in_db_possible_backup_needed_df["links_in_db"].values.tolist())

submission_ids = [ele.split("/")[6] for ele in submission_links_not_uploaded]

backfill_submissions(capture_exceptions=False, to_be_refreshed=submission_ids)

In [ ]:
all_comments_df = []
for sub_id in tqdm(submission_ids):
    comments = pmaw_api.search_submission_comment_ids(
        ids=[sub_id], safe_exit=True, memsafe=True
    )
    
    submission_id_comments = {
    "submission_id": sub_id,
    "comments": [comment for comment in comments],
    }

    all_comments = []
    comments_arr = pmaw_api.search_comments(ids=submission_id_comments["comments"])
    for c in tqdm(comments_arr, leave=False):
        c["submission_id"] = submission_id_comments["submission_id"]
        all_comments.append(c)

    comments_df = pd.DataFrame(all_comments)
    all_comments_df.append(comments_df)
    
all_comments_df = pd.concat(all_comments_df)

In [ ]:
# all_comments_df.iloc[0,].to_dict()
# insert_all_comments_to_db_pmaw(df=all_comments_df)
# update_wsb_comments_status_table()

In [ ]:
### Figure out a way to count number of comments in each submission
## - After each day's insert, make a new table called wsb_comments_diff, which contains date, diff between wsb_comments_tickers/analytics
## - Make a function that will download the difference between the tickers/analytics and create the new bunch of "prepared_statements" and will 
## - Make a view where each submission has it's own submission count with cols = submission_id, no_of_comments
## - Make a smaller table wsb_comments_analytics : contains processed information where each comment_id has it's own ticker info 

In [ ]:
with get_conn() as conn: 
    with conn.cursor() as cur: 
        cur.execute("SELECT EXISTS (SELECT FROM information_schema.tables WHERE table_schema='public' aND table_name='wsb_comments_tickers');")
        wsb_comments_tickers_exists = cur.fetchall()[0][0]

In [ ]:
### Check if these are present in the database, and how many comments do each of these submissions have?
with get_conn() as conn: 
    with conn.cursor() as cur: 
        cur.execute("SELECT created_utc::date as date, submission_id, comment_id FROM wsb_comments_analytics;")
        com_sub_df = pd.DataFrame(cur.fetchall(), columns=["date", "submission_id", "comment_id"])

In [ ]:
agg_df = pd.DataFrame(com_sub_df.groupby(["date", "submission_id"])["comment_id"].nunique())
agg_df = agg_df.sort_index(ascending=True)
agg_df = agg_df.reset_index()
agg_df = agg_df.groupby(["date"]).apply(lambda x: x.sort_values(by="comment_id", ascending=False).max()).drop(columns=["date"])
agg_df.index = pd.to_datetime(agg_df.index)
agg_df["comment_id"].plot(figsize=(20, 10))

In [ ]:
### Over the last 3 months, 
all_business_weeks = pd.period_range(start=agg_df.index[0], end=agg_df.index[-1], freq="W")

all_dates_not_present = []
all_submission_ids_to_update = []
for i in tqdm(range(len(all_business_weeks))):
    date_fmt = "%Y-%m-%d"
    bweek = all_business_weeks[i]
    df_ = agg_df.loc[bweek.start_time: bweek.end_time]
    dates_present = [d for d in pd.to_datetime(df_.index.values)]
    
    dates_not_present = [pd.to_datetime(x) for x in 
                         pd.bdate_range(start=bweek.start_time, end=bweek.end_time) 
                         if x not in dates_present]
    
    ### Which dates are missing?
    all_dates_not_present += dates_not_present

    ### Grab all submission id's that are < -50% of the mean comment_id number 
    relative_df = ( (df_.comment_id - df_.comment_id.max() )/ df_.comment_id.max() ) * 100
    all_submission_ids_to_update += df_.loc[relative_df[relative_df < -50].index, "submission_id"].values.tolist()

In [ ]:
sub_id_exceptions = backfill_submissions(capture_exceptions=True, to_be_refreshed=all_submission_ids_to_update)

sub_id_exceptions = pd.concat(sub_id_exceptions)

sub_id_exceptions.to_csv("sub_id_exceptions.csv", index=True)

In [ ]:
sub_id_exceptions = pd.concat(sub_id_exceptions)

In [ ]:
sub_id_exceptions.to_csv("sub_id_exceptions.csv", index=True)

In [ ]:
sub_id_exceptions = pd.read_csv("sub_id_exceptions.csv", index_col=0)

In [ ]:
insert_all_comments_to_db_pmaw(sub_id_exceptions)

In [ ]:
### Plot WOW counts
# fig, ax = plt.subplots()
# my_dt_fmt = DateFormatter("%Y-%m-%d")
# ax.xaxis.set_major_formatter(my_dt_fmt)

In [ ]:
### For all the links in db, count the number of comments in each of them
# prepared_statements_df = pd.read_csv("prepared_statements.csv")
# prepared_statements_df.head()

### Get some praw data

In [ ]:
from reddit_stuff import get_reddit_client
r_client = get_reddit_client()
api = PushshiftAPI()

In [ ]:
submissions = api.search_submissions(
    search_window=5, 
    max_ids_per_request=1000,
    safe_exit=True, 
    max_results_per_request=100, 
    subreddit="wallstreetbets"
)

all_submissions = [post for post in submissions]

all_submissions

In [ ]:
json.dumps(all_submissions[0])

In [ ]:
# with open("submissions.txt", "w") as f:

In [ ]:
start_epoch = int(datetime.datetime(2019, 1, 1).timestamp())
submissions = api.search_submissions(after=start_epoch, subreddit="wallstreetbets", limit=10000)

In [ ]:
pushshift_host = "https://api.pushshift.io/reddit/submission/comment_ids"

In [ ]:
all_submissions = []
for sub_id in tqdm(submissions):
    all_submissions.append(sub_id)
#     response = requests.get(f"{pushshift_host}/{sub_id}")

In [ ]:
f"{pushshift_host}/{sub_id}"

In [ ]:
sub_id.id

In [ ]:
all_urls = []
all_comment_ids = []
all_submissions_without_comments = []
for sub_id in tqdm(all_submissions):
    all_urls.append(f"{pushshift_host}/{sub_id.id}")
    
#     response = requests.get(f"{pushshift_host}/{sub_id.id}")
#     if response.status_code == 200:
#         data = response.json()["data"]
        
#         if len(data) > 0:
#             all_comment_ids.append(data)
#         else:
#             all_submissions_without_comments.append(sub_id)

In [ ]:
if "get_comment_ids" in locals():
    del get_comment_ids
    
def get_comment_ids(url:str):
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()["data"]

### Get some data

In [ ]:
query = "SELECT id, created_utc, parent_id, body from wsb_comments;"
with get_conn() as conn:
    with conn.cursor() as cur:
        cur.execute(query)
        res = cur.fetchall()
        
df = pd.DataFrame.from_records(res, columns=["id", "created_utc", "parent_id", "body"])

df = df.set_index("id")
df = df.sort_values(by=["created_utc"])
df["created_utc_datetime"] = df["created_utc"].apply(lambda x: datetime.datetime.fromtimestamp(x))
df = df.drop(columns=["created_utc"], inplace=False)
df = df[["created_utc_datetime", "parent_id", "body"]]

In [ ]:
df.head()

In [ ]:
df = df.sort_values(by="created_utc_datetime")

In [ ]:
# nlp = spacy.load('en_core_web_trf')
# emoji = Emoji(nlp)
nlp.add_pipe("emoji", first=True)

In [ ]:
def clean_article(text:str) -> str:
    return text.replace("\n\n", " ").replace("\n", " ").replace("'", "")

In [ ]:
i = 5
text = clean_article(text=df.iloc[i, 2])
doc = nlp(text)
displacy.render(doc, style="ent", jupyter=True)

### Time to rename all tickers as TKR - Use new NER

In [ ]:
query_2 = "SELECT ticker, name, market from ticker_vxes;"

with get_conn() as conn:
    with conn.cursor() as cur:
        cur.execute(query_2)
        tickers_res = cur.fetchall()
        
tickers_df = pd.DataFrame.from_records(tickers_res, columns=["ticker", "name", "market"])
tickers_df = tickers_df[tickers_df["market"] == "stocks"]
tickers_df = tickers_df.sort_values(by="ticker")

tickers_df = tickers_df.reset_index(drop=True)
tickers_df = tickers_df.iloc[2: ]

tickers = tickers_df["ticker"].tolist()

### Create training data for the new NER model

In [ ]:
import os
import json
from multiprocessing import Pool
from sklearn.model_selection import train_test_split

In [ ]:
train_df, test_df = train_test_split(df, test_size=0.33, random_state=42)

In [ ]:
if "find_ner_and_insert_into_db" in locals():
    del find_ner_and_insert_into_db
    
def find_ner_and_insert_into_db(index:str, text:str): 
    text = clean_article(text=text)
    it = {text: {"entities": []}}
    
    for word in text.split(" "):
        word = word.replace(".", " ").replace("'", "").upper()
        if word in tickers:
            it[text]["entities"] += [(m.start(), m.end(), "TKR") for m in re.finditer(word, text)]
                
    insert_query = \
    f"""INSERT INTO wsb_comments_ner(id, ner_labels) VALUES ('{index}', array['{json.dumps(it)}']::json[]) ON CONFLICT (id) DO UPDATE SET ner_labels = EXCLUDED.ner_labels;"""
    with get_conn() as conn:
        with conn.cursor() as cur:
            cur.execute(insert_query)

In [ ]:
df_ = train_df["body"].reset_index()
tasks = [(index, text) for index, text in df_.values]

In [ ]:
# from tqdm.auto import tqdm
import tqdm

In [ ]:
with Pool(processes=os.cpu_count() - 2) as pool:
    for _ in tqdm.tqdm(pool.imap_unordered(find_ner_and_insert_into_db, tasks), total=len(tasks)):
        pass

In [ ]:
# TRAIN_DATA = [
#     ("Who is Shaka Khan?", {"entities": [(7, 17, "PERSON")]}),
#     ("I like London and Berlin.", {"entities": [(7, 13, "LOC"), (18, 24, "LOC")]}),
# ]

# TRAIN_DATA = []

# for i in tqdm(range(len(train_df[["body"]]))):
#     text = clean_article(text=train_df.iloc[i, 2])
#     index = train_df.index[i]

